# Analyse discriminante

Set the work directory. 
> If you don't know, you can use the following code to find out the current working directory: getwd()


In [2]:
setwd("C:/Users/diogo/code/Analyse de Donne/Aula 6/poissons")

In [ ]:
# Importar as bibliotecas necessárias
library(MASS)

### 1º Passo: Preparação dos Dados

In [ ]:
# Nesse caso, os dados já estão disponíveis no ambiente de trabalho através do formato .rda
load("poissons.rda")
poissons

### 2º Passo: Análise Exploratória de Dados (AED)

In [ ]:
poissons2 <- split(poissons[, c(2:15)], poissons[, 1])
poissons2

summary(poissons2$site1)
summary(poissons2$site2)
summary(poissons2$site3)

### 2.b Visualização Gráfica

In [ ]:
boxplot(YEU ~ site, data = poissons, main = "YEU por Site")
boxplot(BR ~ site, data = poissons, main = "BR por Site")
boxplot(FOI ~ site, data = poissons, main = "FOI por Site")

### 3º Passo: Teste de Diferenças entre Grupos

In [ ]:
anova_results <- sapply(poissons[, -1], function(x) {
  summary(aov(x ~ poissons$site))[[1]][["Pr(>F)"]][1]
})

anova_results <- data.frame(
  Variável = names(anova_results),
  p_value = anova_results
)

print(anova_results)

### 4º Passo: Verificação de Multicolinearidade

In [ ]:
library(corrplot)

cor_matrix <- cor(poissons[, -1])

corrplot(cor_matrix, method = "circle")

### 5º Passo: Seleção de Variáveis

In [ ]:
# Opção 1: Seleção manual
modele <- aov(NAG ~ site, data = poissons)
summary(modele)

modele_test <- lda(site ~ ., data = poissons)
modele_test

modele2 <- lda(site ~ YEU + BR, data = poissons)
modele2

prev1 <- lda(site ~ ., data = poissons, CV = TRUE)$class
table(prev1, poissons$site)

sum(prev1 != poissons$site) / nrow(poissons)

# Escolhendo o modelo 2
prev2 <- lda(site ~ YEU + BR, data = poissons, CV = TRUE)$class
table(prev2, poissons$site)
sum(prev2 != poissons$site) / nrow(poissons)

modele2_pred <- predict(modele2, newdata = poissons)$class
modele2_pred

In [ ]:
# Opção 2: Seleção automática com Klar
library(klaR)

preditoras <- poissons[, -1] # Todas as colunas exceto 'site'
resposta <- poissons$site

modelo_step <- stepclass(
  preditoras,
  poissons$site,
  method = "lda",
  direction = "both"
)

formula_lda <- modelo_step$formula
modelo_lda <- lda(formula_lda, data = poissons)
plot(modelo_lda)

### 6º Passo: Validação do Modelo

In [ ]:
predicoes <- predict(modelo_lda, poissons)$class
confusao <- table(Predito = predicoes, Real = resposta)

taxa_acerto <- sum(diag(confusao)) / sum(confusao)
print(paste("Taxa de acerto:", round(taxa_acerto * 100, 2), "%"))

# Validação Cruzada
predicoes_cv <- lda(formula_lda, data = poissons, CV = TRUE)$class
confusao_cv <- table(Predito = predicoes_cv, Real = poissons$site)
taxa_acerto_cv <- sum(diag(confusao_cv)) / sum(confusao_cv)
print(paste("Taxa de acerto com validação cruzada:", round(taxa_acerto_cv * 100, 2), "%"))

### 7º Passo: Entendimento dos Dados

In [ ]:
print("Coeficientes das funções discriminantes:")
print(modelo_lda$scaling)

# Teste de Shapiro-Wilk para NAG e YEU
print("Teste de Shapiro-Wilk para NAG:")
by(poissons$NAG, poissons$site, shapiro.test)

print("Teste de Shapiro-Wilk para YEU:")
by(poissons$YEU, poissons$site, shapiro.test)

### 8º Passo: Simulação de Novo Dado

In [ ]:
n_don1 <- matrix(
  c(10, 70, 50, 100, 20, 120, 130, 1, 120, 200, 50, 50, 18, 10),
  ncol = 14,
  byrow = TRUE
)
n_don1 <- data.frame(n_don1)
names(n_don1) <- names(poissons[-1])
n_don1
predict(modele2, newdata = n_don1)

plot(modele)